In [5]:
#!pip install tqdm
import torchvision.models as models
import sys
import torch
from torch import nn
import time
from tqdm import tqdm
import torch.optim as optim
import copy

# Specify where to find the data preparation class
sys.path.append('../../Data_Preparation')
from Preparation import CustomDataLoader

In [6]:
# VGG16 training data (ImageNet) properties
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]
DIMENSIONS = 3
#SIZE = 256

LR = 0.1
MOMENTUM=0.85
WEIGHT_DECAY = 1e-4
LR_STEP_SIZE = 5
LR_GAMMA = 0.1

In [7]:
# Instantiate the CustomDataLoader class for training
train_data_loader = CustomDataLoader(data_path="../../FER2013_Data", batch_size=16, dataset_type="train", mean=MEAN, std=STD, dimensions=DIMENSIONS).data_loader
test_data_loader = CustomDataLoader(data_path="../../FER2013_Data", batch_size=16, dataset_type="test", mean=MEAN, std=STD, dimensions=DIMENSIONS).data_loader

# Confirm correct data load
print("Train Data Loader:")
for batch_idx, (inputs, labels) in enumerate(train_data_loader):
    print("Batch Index:", batch_idx)
    print("Inputs Shape:", inputs.shape)
    print("Labels Shape:", labels.shape)
    # Print the first few labels in the batch
    print("Labels:", labels[:5])
    # Break after printing a few batches
    if batch_idx == 2:
        break

Train Data Loader:
Batch Index: 0
Inputs Shape: torch.Size([16, 3, 299, 299])
Labels Shape: torch.Size([16])
Labels: tensor([6, 2, 6, 0, 6])
Batch Index: 1
Inputs Shape: torch.Size([16, 3, 299, 299])
Labels Shape: torch.Size([16])
Labels: tensor([2, 0, 2, 3, 0])
Batch Index: 2
Inputs Shape: torch.Size([16, 3, 299, 299])
Labels Shape: torch.Size([16])
Labels: tensor([0, 2, 0, 0, 3])


In [8]:
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
vgg19 = models.vgg19(pretrained=True)


C:\Users\kh2197vi\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\kh2197vi\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to C:\Users\kh2197vi/.cache\torch\hub\checkpoints\vgg19-dcbb9e9d.pth
100.0%


In [9]:
for param in vgg19.parameters():
    param.requires_grad = False
dataloaders = {'train':train_data_loader, 'val':test_data_loader}


In [10]:
num_classes = len(train_data_loader.dataset.classes)
vgg19.classifier[6] = nn.Linear(4096, num_classes)
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(
    vgg19.parameters(),
    lr=LR,
    momentum=MOMENTUM,
    weight_decay=WEIGHT_DECAY
)

LR_SCHEDULER = torch.optim.lr_scheduler.StepLR(optimizer, step_size=LR_STEP_SIZE, gamma=LR_GAMMA)

optimizer_vgg19 = optim.Adam(vgg19.parameters())
num_epochs = 25
for epoch in tqdm(range(num_epochs)):
    print('Epoch {}/{}'.format(epoch,num_epochs -1))
    print('-'*10)
    total_batch = len(train_data_loader.dataset) // 16
    for phase in ['train', 'val']:
        if phase == 'train':
            vgg19.train()
        else:
            vgg19.eval()

        running_loss = 0.0
        corrects = 0

        for i,(inputs, labels) in enumerate(dataloaders[phase]):
            optimizer_vgg19.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs_vgg19 = vgg19(inputs)
                _, preds_vgg19 = torch.max(outputs_vgg19, 1)

                loss_vgg19 = criterion(outputs_vgg19, labels)

                if phase == 'train':
                    loss_vgg19.backward()
                    optimizer_vgg19.step()

                    if (i+1) % 5 == 0:
                        print('Epoch [%d/%d], lter [%d/%d] Loss: %.4f'
                             %(epoch+1, num_epochs, i+1, total_batch, loss_vgg19.item()))

            running_loss += loss_vgg19.item() * inputs.size(0)
            corrects += torch.sum(preds_vgg19 == labels.data)

        epoch_loss = running_loss / len(dataloaders[phase])
        epoch_acc = corrects.double() / len(dataloaders[phase])

        LR_SCHEDULER.step()

        print('{} Loss: {:.4f} | Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

  0%|          | 0/25 [00:00<?, ?it/s]

Epoch 0/24
----------
Epoch [1/25], lter [5/1752] Loss: 1.9588
Epoch [1/25], lter [10/1752] Loss: 1.7824
Epoch [1/25], lter [15/1752] Loss: 2.0534
Epoch [1/25], lter [20/1752] Loss: 2.5989
Epoch [1/25], lter [25/1752] Loss: 1.8659
Epoch [1/25], lter [30/1752] Loss: 1.5633
Epoch [1/25], lter [35/1752] Loss: 1.5314
Epoch [1/25], lter [40/1752] Loss: 1.7160
Epoch [1/25], lter [45/1752] Loss: 1.9797
Epoch [1/25], lter [50/1752] Loss: 2.1457
Epoch [1/25], lter [55/1752] Loss: 1.5537
Epoch [1/25], lter [60/1752] Loss: 1.6662
Epoch [1/25], lter [65/1752] Loss: 1.6317
Epoch [1/25], lter [70/1752] Loss: 1.6429
Epoch [1/25], lter [75/1752] Loss: 1.7765
Epoch [1/25], lter [80/1752] Loss: 1.6707
Epoch [1/25], lter [85/1752] Loss: 1.9501
Epoch [1/25], lter [90/1752] Loss: 2.1365
Epoch [1/25], lter [95/1752] Loss: 1.9167
Epoch [1/25], lter [100/1752] Loss: 1.9768
Epoch [1/25], lter [105/1752] Loss: 1.8830
Epoch [1/25], lter [110/1752] Loss: 2.0005
Epoch [1/25], lter [115/1752] Loss: 1.6972
Epoch [1/

C:\Users\kh2197vi\AppData\Roaming\Python\Python312\site-packages\torch\optim\lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


train Loss: 27.9394 | Acc: 5.3132


  4%|▍         | 1/25 [1:11:34<28:37:44, 4294.36s/it]

...

val Loss: 24.6728 | Acc: 6.6169
Epoch 23/24
----------
Epoch [24/25], lter [5/1752] Loss: 2.0895
Epoch [24/25], lter [10/1752] Loss: 1.6369
Epoch [24/25], lter [15/1752] Loss: 1.7204
Epoch [24/25], lter [20/1752] Loss: 2.2551
Epoch [24/25], lter [25/1752] Loss: 2.2986
Epoch [24/25], lter [30/1752] Loss: 1.3375
Epoch [24/25], lter [35/1752] Loss: 1.3055
Epoch [24/25], lter [40/1752] Loss: 1.8158
Epoch [24/25], lter [45/1752] Loss: 1.3859
Epoch [24/25], lter [50/1752] Loss: 1.9786
Epoch [24/25], lter [55/1752] Loss: 1.6712
Epoch [24/25], lter [60/1752] Loss: 1.8491
Epoch [24/25], lter [65/1752] Loss: 1.4285
Epoch [24/25], lter [70/1752] Loss: 1.6152
Epoch [24/25], lter [75/1752] Loss: 1.5682
Epoch [24/25], lter [80/1752] Loss: 1.3664
Epoch [24/25], lter [85/1752] Loss: 2.0191
Epoch [24/25], lter [90/1752] Loss: 1.6135
Epoch [24/25], lter [95/1752] Loss: 1.3347
Epoch [24/25], lter [100/1752] Loss: 1.3084
Epoch [24/25], lter [105/1752] Loss: 1.6643
Epoch [24/25], lter [110/1752] Loss: 1.70

 96%|█████████▌| 24/25 [28:54:26<1:01:56, 3716.95s/it]

val Loss: 25.5638 | Acc: 6.3452
Epoch 24/24
----------
Epoch [25/25], lter [5/1752] Loss: 1.6570
Epoch [25/25], lter [10/1752] Loss: 2.0992
Epoch [25/25], lter [15/1752] Loss: 1.6721
Epoch [25/25], lter [20/1752] Loss: 2.0053
Epoch [25/25], lter [25/1752] Loss: 1.2624
Epoch [25/25], lter [30/1752] Loss: 2.2772
Epoch [25/25], lter [35/1752] Loss: 1.5875
Epoch [25/25], lter [40/1752] Loss: 1.9600
Epoch [25/25], lter [45/1752] Loss: 1.3293
Epoch [25/25], lter [50/1752] Loss: 1.4838
Epoch [25/25], lter [55/1752] Loss: 1.5880
Epoch [25/25], lter [60/1752] Loss: 1.8579
Epoch [25/25], lter [65/1752] Loss: 1.7651
Epoch [25/25], lter [70/1752] Loss: 1.9794
Epoch [25/25], lter [75/1752] Loss: 1.3302
Epoch [25/25], lter [80/1752] Loss: 1.9040
Epoch [25/25], lter [85/1752] Loss: 1.6330
Epoch [25/25], lter [90/1752] Loss: 1.7632
Epoch [25/25], lter [95/1752] Loss: 1.7877
Epoch [25/25], lter [100/1752] Loss: 1.9142
Epoch [25/25], lter [105/1752] Loss: 1.8172
Epoch [25/25], lter [110/1752] Loss: 1.48

100%|██████████| 25/25 [29:42:30<00:00, 4278.01s/it]  

val Loss: 25.4271 | Acc: 6.3430


In [11]:
# Saving the model
torch.save(vgg19.state_dict(), 'trained_vgg19.pt')

In [12]:
# Initialize the correct_predictions and total_predictions variables
correct_predictions, total_predictions = 0, 0

# Loop over the batches of the test set
for batch_data in dataloaders['val']:
    # Forward pass
    inputs, target = batch_data
    output = vgg19(inputs)

    # Get the predicted class
    _, predicted = torch.max(output.data, 1)

    # Increase the total predictions counter
    total_predictions += target.size(0)

    # Check if the predicted class is equal to the true class
    correct_predictions += (predicted == target).sum().item()

# Calculate the accuracy
accuracy = correct_predictions / total_predictions
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 40.02%


In [13]:
# Initialize the correct_predictions and total_predictions variables
correct_predictions, total_predictions = 0, 0

# Loop over the batches of the test set
for batch_data in dataloaders['train']:
    # Forward pass
    inputs, target = batch_data
    output = vgg19(inputs)

    # Get the predicted class
    _, predicted = torch.max(output.data, 1)

    # Increase the total predictions counter
    total_predictions += target.size(0)

    # Check if the predicted class is equal to the true class
    correct_predictions += (predicted == target).sum().item()

# Calculate the accuracy
accuracy = correct_predictions / total_predictions
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 42.39%
